**Tips: 导出到csv文件**

dataframe可以使用to_csv方法方便地导出到csv文件中，如果数据中含有中文，一般encoding指定为”utf-8″,否则导出时程序会因为不能识别相应的字符串而抛出异常，index指定为False表示不用导出dataframe的index数据。

df.to_csv(file_path, encoding='utf-8', index=False)

In [27]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import time
matplotlib.style.use('ggplot')
%pylab inline

Populating the interactive namespace from numpy and matplotlib


# 读取数据

### 数据格式-用户行为表（mars_tianchi_user_actions）

| name | data_type | describe | example |
| ------ | ----------- | --------- | ---------- |
| user_id | String | 用户唯一标识 | 7063b3d0c075a4d276c5f06f4327cf4a |
| song_id | String | 歌曲唯一标识 | effb071415be51f11e845884e67c0f8c |
| gmt_create | String | 用户播放时间（unix时间戳表示）精确到小时 | 1426406400 |
| action_type | String | 行为类型：1，播放；2，下载，3，收藏 | 1 |
| Ds | String | 记录收集日（分区） | 20150315 |

In [3]:
# reading user actions data into pandas.DataFrame df_user_actions
df_user_actions = pd.read_csv('./data/p2/p2_mars_tianchi_user_actions.csv', 
                              header = None, 
                              names = ('user_id', 'song_id', 'gmt_create', 'action_type', 'Ds'))

In [8]:
# show the data of first 5 lines.
# df_user_actions.head()

,user_id,song_id,gmt_create,action_type,Ds
0,7063b3d0c075a4d276c5f06f4327cf4a,effb071415be51f11e845884e67c0f8c,1426406400,1,20150315
1,0db66c0dd3993fd3504bb98c3beb15b3,f87ff481d85d2f95335ab602f38a7655,1426417200,1,20150315
2,f8c065dc140ec74c6e44144164e618e3,8a27d9a6c59628c991c154e8d93f412e,1426406400,2,20150315
3,2c6082cf0d68e244f2a10325e8d1b85b,ecea5fe33e6817d09c395f2910479728,1426417200,1,20150315
4,2c6082cf0d68e244f2a10325e8d1b85b,31a3d0420d89c9b121bb55dbdbbeda6b,1426417200,1,20150315


In [15]:
# count the unique id number of user & song
print 'In user_action dataset, user id count: ', len(set(df_user_actions['user_id']))
print 'In user_action dataset, song id count: ', len(set(df_user_actions['song_id']))

In user_action dataset, user id count:  536024
In user_action dataset, song id count:  24943


### 数据格式-歌曲艺人（mars_tianchi_songs）

| name | data_type | describe | example |
| ------ | ----------- | --------- | ---------- |
| song_id | String | 歌曲唯一标识 | c81f89cf7edd24930641afa2e411b09c |
| artist_id | String | 歌曲所属的艺人Id | 03c6699ea836decbc5c8fc2dbae7bd3b |
| publish_time | String | 歌曲发行时间，精确到天 | 20150325 |
| song_init_plays | String | 歌曲的初始播放数，表明该歌曲的初始热度 | 0 |
| Language | String | 数字表示1,2,3… | 100 |
| Gender | String | 1,2,3 | 1 |

In [12]:
# reading song info data into pandas.DataFrame df_song_info
df_song_info = pd.read_csv('./data/p2/p2_mars_tianchi_songs.csv', 
                           header = None, 
                           names = ('song_id', 'artist_id', 'public_time', 'popular', 'language', 'gender'))

In [11]:
# show the data of first 5 lines.
# df_song_info.head()

,song_id,artist_id,public_time,popular,language,gender
0,c81f89cf7edd24930641afa2e411b09c,03c6699ea836decbc5c8fc2dbae7bd3b,20150325,0,100,1
1,c0d7130777c1f1c417e78646946ed909,03c6699ea836decbc5c8fc2dbae7bd3b,20150325,0,100,1
2,200c9131cf929bab418d380356be5f42,03c6699ea836decbc5c8fc2dbae7bd3b,20150325,0,100,1
3,78fedfdf13fc820e363e39986ff91e94,03c6699ea836decbc5c8fc2dbae7bd3b,20110910,1717,1,1
4,95b99faf432d33772d63f828bf2d0921,03c6699ea836decbc5c8fc2dbae7bd3b,20110910,434,1,1


In [16]:
# count the unique id number of song & artist
print 'In song_info dataset, song id count: ', len(set(df_song_info['song_id']))
print 'In song_info dataset, artist id count: ', len(set(df_song_info['artist_id']))

In song_info dataset, song id count:  26958
In song_info dataset, artist id count:  100


**Tips：DataFrame切片处理**

DataFrame 对象的标准切片语法为：.ix[::,::]。ix 对象可以接受两套切片，分别为行（axis=0）和列（axis=1）的方向

不使用 ix时：

索引时，选取的是列
切片时，选取的是行

# 数据整理

In [20]:
# generate the dict of artist and their songs.
artists = {}
for i in range(df_song_info.shape[0]):
    artist = df_song_info.iloc[i]['artist_id']
    song = df_song_info.iloc[i]['song_id']
    if artist not in artists:
        artists[artist] = []
    artists[artist].append(song)
# generate the list of all songs.
songs = list(set(df_song_info['song_id']))

**统计每首歌的每日播放量**

In [34]:
# get the date list in dataset.
date_past =  list(set(df_user_actions['Ds']))
date_past.sort()
# record the start time
time_start = time.asctime(time.localtime(time.time()))
print 'start processing at', time_start
# open file
fp = open('./data/p2/songs_by_day.csv', 'w')

for song in songs:
    data = df_user_actions[df_user_actions.song_id == song]
    content = ''
    for date in date_past:
        song_today = data[data.Ds == date]
        play = len(song_today[song_today.action_type == 1])
        download = len(song_today[song_today.action_type == 2])
        like = len(song_today[song_today.action_type == 3])
        content += "%s,%d,%d,%d,%d\n" % (song,date,play,download,like)
    fp.write(content)
fp.close()

# record the end time
time_end = time.asctime(time.localtime(time.time()))
print 'end processing at', time_end

start processing at Tue Jun  7 14:59:56 2016


KeyboardInterrupt: 

**统计每个歌手的每日播放量**

In [12]:
# read the analyse by day into pandas.DataFrame df_songs_by_day
df_songs_by_day = pd.read_csv('./data/p2/songs_by_day.csv', 
                              header = None, 
                              names = ('song_id', 'date', 'play', 'download', 'like'))

In [13]:
# show data property
df_songs_by_day.describe()

,date,play,download,like
count,1.984086e+06,1.984086e+06,1.984086e+06,1.984086e+06
mean,2.015056e+07,2.410986e+00,4.094717e-01,2.832639e-02
std,1.709344e+02,2.103799e+01,3.332696e+00,3.560573e-01
min,2.015030e+07,0.000000e+00,0.000000e+00,0.000000e+00
25%,2.015042e+07,0.000000e+00,0.000000e+00,0.000000e+00
50%,2.015053e+07,0.000000e+00,0.000000e+00,0.000000e+00
75%,2.015072e+07,0.000000e+00,0.000000e+00,0.000000e+00
max,2.015083e+07,3.767000e+03,7.080000e+02,1.160000e+02


In [14]:
# show the first 5 lines
df_songs_by_day.head()

,song_id,date,play,download,like
0,b4e23c8cca36326a787763652bdcfb4f,20150301.0,0.0,1.0,0.0
1,b4e23c8cca36326a787763652bdcfb4f,20150302.0,0.0,0.0,0.0
2,b4e23c8cca36326a787763652bdcfb4f,20150303.0,0.0,0.0,0.0
3,b4e23c8cca36326a787763652bdcfb4f,20150304.0,0.0,0.0,0.0
4,b4e23c8cca36326a787763652bdcfb4f,20150305.0,0.0,0.0,0.0
5,b4e23c8cca36326a787763652bdcfb4f,20150306.0,0.0,0.0,0.0
6,b4e23c8cca36326a787763652bdcfb4f,20150307.0,0.0,0.0,0.0
7,b4e23c8cca36326a787763652bdcfb4f,20150308.0,0.0,0.0,0.0
8,b4e23c8cca36326a787763652bdcfb4f,20150309.0,0.0,0.0,0.0
9,b4e23c8cca36326a787763652bdcfb4f,20150310.0,0.0,0.0,0.0


In [20]:
# analyse the data by artist
date_past_new = pd.date_range('20150301', '20150830')
for artist in artists:
    temp = df_songs_by_day[['play', 'download', 'like']][df_songs_by_day.song_id == artists[artist][0]]
    temp = temp[['play', 'download', 'like']]
    temp['newdate'] = date_past_new
    temp = temp.set_index(['newdate'])
    if len(artists[artist]) > 1:
        for song in artists[artist][1:]:
            temp2 = df_songs_by_day[['play', 'download', 'like']][df_songs_by_day.song_id == song]
            temp2['newdate'] = date_past_new
            temp2 = temp2.set_index(['newdate'])
            temp = temp.add(temp2)
    temp['artist_id'] = artist
    temp['date'] = date_past
    temp[['artist_id', 'date', 'play', 'download', 'like']].to_csv('./data/p2/artist_by_day.csv', mode = 'a', encoding='utf-8', header = False, index = False)
print 'done!'

done!


# 生成训练集&数据集

In [7]:
df_artists = pd.read_csv('./data/artists.csv', header = None, names = ['date', 'play', 'download', 'like', 'artist_id'])

In [10]:
df_artist_1 = df_artists[df_artists['artist_id'] == 'e087f8842fe66efa5ccee42ff791e0ca']

In [ ]:
df_artist_1 = df_artist_1[['']]

In [22]:
from sklearn import preprocessing

In [48]:
import csv
reader=csv.reader(open("artist_1.csv","rb"),delimiter=',')
x=list(reader)
result=np.array(x).astype('float')

ValueError: invalid literal for float(): 2015-03-01

In [36]:
X = result[:, 3:5]

In [76]:
for i in range(0, X.shape[0]):
    for j in range(0, X.shape[1]):
        X[i][j] = int(X[i][j])

In [78]:
type(X[0][0])

numpy.string_